In [8]:
import torch
from torch.utils.data import DataLoader
import os
import pandas as pd
from utils.dataset import PandasDataset, MultiColorSpaceTransform
from utils.models import EfficientNet
from utils.metrics import evaluation, format_metrics
import albumentations

In [9]:
output_dimensions = 5
data_dir = '../../../dataset'
ROOT_DIR = "../.."
images_dir = os.path.join(data_dir, 'tiles')
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
df_test = pd.read_csv(f"../../data/test.csv")
backbone_model = 'efficientnet-b0'
pretrained_model = {
    backbone_model: f'{ROOT_DIR}/efficientnet-b0-08094119.pth'
}

In [10]:
print("Cuda", device)

Cuda cuda


In [11]:
val_transform =albumentations.Compose([
    MultiColorSpaceTransform()
])

In [12]:
dataloader = DataLoader(
    PandasDataset(images_dir, df_test, transforms=val_transform),
    batch_size=2,
    shuffle=False,
)

In [14]:
model = EfficientNet(
    backbone=backbone_model,
    output_dimensions=output_dimensions,
    pre_trained_model=pretrained_model
)
model.to(device)
model.load_state_dict(
    torch.load(
        f"{ROOT_DIR}/models/with-noise-mult-color.pth",
        weights_only=True
    )
)

Loaded pretrained weights for efficientnet-b0


RuntimeError: Error(s) in loading state_dict for EfficientNet:
	size mismatch for efficient_net._conv_stem.weight: copying a param with shape torch.Size([32, 18, 3, 3]) from checkpoint, the shape in current model is torch.Size([32, 3, 3, 3]).

In [ ]:
response = evaluation(model, dataloader, device)
result = format_metrics(response[0])
print(result)